In [1]:
import pandas as pd

In [2]:
'''
IMPORT TRAIN DATA
'''
data = pd.read_csv('train.csv')

print('Length of train data: ', len(data))
data.head(5)

Length of train data:  891


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
'''
IMPORT VALIDATION DATA - For Kaggle Submission
'''
val_data = pd.read_csv('test.csv')

print('Length of validation data: ', len(val_data))

Length of validation data:  418


In [4]:
'''
DATA EXPLORATION
'''

data_exploration = data.copy()

# distribution of passengers who survived / did not survive
# 0 = did not survive, 1 = survived
count_not_survived = data_exploration['Survived'].value_counts()[0]
count_survived = data_exploration['Survived'].value_counts()[1]

# distribution of passengers by gender
count_male = data_exploration['Sex'].value_counts()[0]
count_female = data_exploration['Sex'].value_counts()[1]

# distribution of passengers by survived AND gender
df_survived_gender = data_exploration.groupby(['Survived', 'Sex']).size().reset_index(name = 'Count')
count_male_survived = df_survived_gender[(df_survived_gender['Survived'] == 1) & (df_survived_gender['Sex'] == 'male')]['Count'].iloc[0]
count_male_not_survived = df_survived_gender[(df_survived_gender['Survived'] == 0) & (df_survived_gender['Sex'] == 'male')]['Count'].iloc[0]
count_female_survived = df_survived_gender[(df_survived_gender['Survived'] == 1) & (df_survived_gender['Sex'] == 'female')]['Count'].iloc[0]
count_female_not_survived = df_survived_gender[(df_survived_gender['Survived'] == 0) & (df_survived_gender['Sex'] == 'female')]['Count'].iloc[0]

# distribution of passengers by survived AND age
df_age_survived = pd.crosstab(pd.cut(data_exploration['Age'], bins = 10), data_exploration['Survived'])

# distribution of passengers by survived AND fare
df_fare_survived = pd.crosstab(pd.cut(data_exploration['Fare'], bins = 10), data_exploration['Survived'])

# distribution of passengers by survived AND Pclass (socio-economic class)
df_pclass_survived = data_exploration.groupby(['Pclass', 'Survived']).size().reset_index(name = 'Count')

# distribution of passengers by survived AND Parch (# of parents/children onboard)
df_parch_survived = data_exploration.groupby(['Parch', 'Survived']).size().reset_index(name = 'Count')

# distribution of passengers by survived AND SibSp (# of siblings spouse on board)
df_sibsp_survived = data_exploration.groupby(['SibSp', 'Survived']).size().reset_index(name = 'Count')

print(
'''

STATISTICS:

Number of rows in train data: {}

Number of NA in all columns:
{}


(1) BY GENDER

<1>
Number of passengers who survived: {}
Number of passengers who did not survive: {}

<2>
Number of male passengers: {}
Number of female passengers: {}

<3>
Number of male passengers who survived: {}
Number of male passengers who did not survive: {}
Number of female passengers who survived: {}
Number of female passengers who did not survived: {}


(2) BY AGE

{}


(3) BY PASSENGER FARE

{}


(4) BY PCLASS

{}


(5) BY # OF PARENTS/CHILDREN

{}


(5) BY # OF SIBLINGS / SPOUSES

{}


'''.format(len(data_exploration), data_exploration.isna().sum(),
           count_survived, count_not_survived, 
           count_male, count_female, 
           count_male_survived, count_male_not_survived,
           count_female_survived, count_female_not_survived, 
           df_age_survived,
           df_fare_survived,
           df_pclass_survived,
           df_parch_survived,
           df_sibsp_survived))



STATISTICS:

Number of rows in train data: 891

Number of NA in all columns:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


(1) BY GENDER

<1>
Number of passengers who survived: 342
Number of passengers who did not survive: 549

<2>
Number of male passengers: 577
Number of female passengers: 314

<3>
Number of male passengers who survived: 109
Number of male passengers who did not survive: 468
Number of female passengers who survived: 233
Number of female passengers who did not survived: 81


(2) BY AGE

Survived            0   1
Age                      
(0.34, 8.378]      18  36
(8.378, 16.336]    27  19
(16.336, 24.294]  114  63
(24.294, 32.252]  104  65
(32.252, 40.21]    66  52
(40.21, 48.168]    46  24
(48.168, 56.126]   24  21
(56.126, 64.084]   15   9
(64.084, 72.042]    9   0
(72.04

In [5]:
'''
FEATURE ENGINEERING
'''

def feature_engineering(df):
    # drop passengerid, Name (identifiers), cabin (mostly nans), ticket, embarked (low relevance to survived) columns
    df = df.drop(['PassengerId', 'Name', 'Cabin', 'Ticket', 'Embarked'], axis = 1)

    # convert age to categorical type using classification 
    # ref: https://www.statcan.gc.ca/en/concepts/definitions/age2
    # children (1), youth (2), adults (3), seniors (4), nans (0)
    df['Age'] = pd.to_numeric(pd.cut(df['Age'], bins = [0, 14, 24, 64, 120], labels = [1, 2, 3, 4]))
    df['Age'] = df['Age'].fillna(0)
    #data_feature_eng['Age'].replace({1: "Children", 2: "Youth", 3: "Adults", 4: "Seniors", 0: "No Data"}, inplace = True)

    # convert sibsp, parch to binary (whether siblings/spouse, parent/children is onboard (1) OR not (0))
    df['SibSp'][df['SibSp'] == 0] = 0
    df['SibSp'][df['SibSp'] >= 1] = 1
    df['Parch'][df['Parch'] == 0] = 0
    df['Parch'][df['Parch'] >= 1] = 1

    # convert passenger fare to first (>= $30), second (>= $13, <= 29), third (<= $12) class
    # ref: https://www.bbc.co.uk/bitesize/topics/z8mpfg8/articles/zng8jty
    df['Fare'][df['Fare'] < 13] = 3
    df['Fare'][(df['Fare'] >= 13) & (df['Fare'] < 30)] = 2
    df['Fare'][df['Fare'] >= 30] = 1

    # convert sex column to binary (male (1), female (0))
    df['Sex'][df['Sex'] == 'male'] = 1
    df['Sex'][df['Sex'] == 'female'] = 0

    # remove Fare column since there is multicollinearity issues between Fare and Pclass columns
    df = df.drop(['Fare'], axis = 1)
    return df

train_data_feature_eng = data.copy()
train_data_feature_eng = feature_engineering(train_data_feature_eng)

print('Number of rows: ', len(train_data_feature_eng))
train_data_feature_eng.head(20)

Number of rows:  891


C:\Users\JINGXU~1.NG\AppData\Local\Temp/ipykernel_21988/3622774351.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SibSp'][df['SibSp'] == 0] = 0
C:\Users\JINGXU~1.NG\AppData\Local\Temp/ipykernel_21988/3622774351.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SibSp'][df['SibSp'] >= 1] = 1
C:\Users\JINGXU~1.NG\AppData\Local\Temp/ipykernel_21988/3622774351.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Parch']

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,1,2.0,1,0
1,1,1,0,3.0,1,0
2,1,3,0,3.0,0,0
3,1,1,0,3.0,1,0
4,0,3,1,3.0,0,0
5,0,3,1,0.0,0,0
6,0,1,1,3.0,0,0
7,0,3,1,1.0,1,1
8,1,3,0,3.0,0,1
9,1,2,0,1.0,1,0


In [6]:
'''
LOGISTIC REGRESSION MODEL DEVELOPMENT
'''

train_data_model = train_data_feature_eng.copy()

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

feature_cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch']

X = train_data_model[feature_cols]
y = train_data_model['Survived']

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

# fitting logistic regression model with train data
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# prediction for test data
y_pred = logreg.predict(X_test)

# confusion matrix to get accuracy of prediction
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))

Accuracy:  0.7802690582959642
Precision:  0.7108433734939759
Recall:  0.7023809523809523


In [7]:
'''
DATA VALIDATION USING UNSEEN DATA 
'''

val_data_feature_eng = val_data.copy()

# apply same feature engineering steps to validation data
val_data_feature_eng = feature_engineering(val_data_feature_eng)

# predicted Survived on unseen data using trained Logs Regression model
y_val = logreg.predict(val_data_feature_eng)
y_val = y_val.tolist()

print('Length of y_val: ', len(y_val))

Length of y_val:  418


C:\Users\JINGXU~1.NG\AppData\Local\Temp/ipykernel_21988/3622774351.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SibSp'][df['SibSp'] == 0] = 0
C:\Users\JINGXU~1.NG\AppData\Local\Temp/ipykernel_21988/3622774351.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SibSp'][df['SibSp'] >= 1] = 1
C:\Users\JINGXU~1.NG\AppData\Local\Temp/ipykernel_21988/3622774351.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Parch']

In [8]:
'''
PREPARING PREDICTIONS FOR SUBMISSION
'''
val_data_submission = val_data.copy()

val_data_submission = val_data_submission[['PassengerId']]
val_data_submission['Survived'] = y_val
val_data_submission.to_csv('kaggle_titanic_logs_reg_submission.csv', index = False)